# Extract surnames from church books index pages

In [4]:
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw

## Load annotated index pages 
Data annotated on transkribus and exported to xml

In [20]:
xml_filenames = ["transkribus_data/ground_truth/Braian_Pages/Braian_Pages/page/eaa1216_001_0000275_00336_m.xml",
                 "transkribus_data/ground_truth/Braian_Pages/Braian_Pages/page/eaa1216_001_0000275_00337_m.xml",
                 "transkribus_data/ground_truth/Aitor_Pages_transkribus_/Aitor_Pages_transkribus/page/eaa1216_001_0000275_00338_m.xml",
                 "transkribus_data/ground_truth/Aitor_Pages_transkribus_/Aitor_Pages_transkribus/page/eaa1216_001_0000275_00339_m.xml",
#                  "transkribus_data/ground_truth/index_pages_ML/index_pages_ML/page/eaa1216_001_0000275_00340_m.xml",
#                  "transkribus_data/ground_truth/index_pages_ML/index_pages_ML/page/eaa1216_001_0000275_00341_m.xml",
                 "transkribus_data/ground_truth/Iqbal_Pages_transkribus/Iqbal_Pages_transkribus/page/eaa1216_001_0000275_00342_m.xml",
                 "transkribus_data/ground_truth/Iqbal_Pages_transkribus/Iqbal_Pages_transkribus/page/eaa1216_001_0000275_00343_m.xml"]

external_surnames_db = "databases/Estonian Surname Database.txt"

## Parse xml files

In [22]:
annotations = []

for xml_filename in xml_filenames:
    with open(xml_filename, "r") as xml_file:
        xml_content = xml_file.read()
        soup = BeautifulSoup(xml_content, "lxml")

        text_lines = soup.find_all("textline")
        for data in text_lines:
            polygon = []
            polygon_text = data.find('coords').get('points')        
            coord = polygon_text.split(" ")
            for xy in coord:
                point = xy.split(",")
                polygon.append((int(point[0]), int(point[1])))
            # Get only text from element "TextEquiv". avoid "Word" Tag
            for ch in data.children:
                if ch.name == "textequiv":
                    content = ch.find('unicode').get_text()
#             print(polygon, content)
            annotations.append({'polygon': polygon, 'content': content})

In [23]:
indexed_names = [ann["content"] for ann in annotations]
len(indexed_names)

737

## Load additional list of surnames

In [29]:
with open(external_surnames_db, "r") as file:
    external_surnames = file.readlines()

In [30]:
external_surnames = [surnames.replace("\n", "") for surnames in external_surnames]
external_surnames = [surnames.replace(" ", "") for surnames in external_surnames]
len(external_surnames)

300

## Merge all available data

In [31]:
full_surnames_db = indexed_names + external_surnames
full_surnames_db.sort()
len(full_surnames_db)

1037

In [28]:
with open("databases/final_database_estonian_surnames.txt", "w") as file:
    file.write('\n'.join(full_surnames_db))